# Load packages

In [58]:
import Astroshaper
import GLMakie
import Downloads
import Statistics
using StaticArrays
using LinearAlgebra

# Download files

In [7]:
path_obj = "SHAPE_SFM_49k_v20180804.obj"
if !isfile(path_obj)
    url_obj = "https://data.darts.isas.jaxa.jp/pub/hayabusa2/paper/Watanabe_2019/SHAPE_SFM_49k_v20180804.obj"
    Downloads.download(url_obj, path_obj)
end

# Load shape model

In [75]:
path_jld = splitext(path_obj)[1]*".jld2"
if isfile(path_jld)
    shape = Astroshaper.ShapeModel(path_jld; scale=1000, find_visible_facets=true, save_shape=true)
    println("A preprocessed shape file ($path_jld) was loaded.")
else
    shape = Astroshaper.ShapeModel(path_obj; scale=1000, find_visible_facets=true, save_shape=true)
    println("A shape file ($path_obj) was loaded.")
end

A preprocessed shape file (SHAPE_SFM_49k_v20180804.jld2) was loaded.


# Replicate the result of Kanamaru et al. (2021)
Thermophysical model in Kanamaru et al. (2021):
- 3D shape of asteroid Ryugu
- Self-shadowing
- Zero-conductivity
- Ignore reabsorption of scttering and radiation

In [76]:
"""
    getSolarIrradiation(distance) -> solar_irrad

Calculate the solar irradiation on the body

# Parameter
- `rₕ` : heliocentric distance of the body [m]

# Return
- `F☉` : solar irradiation [W/m^2]
"""
getSolarIrradiation(rₕ) = Astroshaper.SOLAR_CONST / (rₕ / Astroshaper.AU)^2


"""
    getSolarCondition(orbit, spin, time) -> Φ, r_sun

Get the solar irradition and the direction of the Sun

# Parameters
- `orbit` : Orbital elements
- `spin`  : Spin parameters
- `time`  : Epoch in seconds

# Returns
- `F☉` : solar irradiation [W/m^2]
- `r̂☉` : solar direction in the body-fixed frame
"""
function getSolarCondition(orbit, spin, time)
    u = Astroshaper.solveKeplerEquation2(orbit, time)
    r = Astroshaper.get_r(orbit, u)
    F☉ = getSolarIrradiation(norm(r))

    r̂☉ = normalize(r) * -1  # Shift the origin from the sun to the body

    spin_phase = spin.ω * time
    r̂☉ = Astroshaper.orbit_to_body(r̂☉, spin.γ, spin.ε, spin_phase)

    F☉, r̂☉
end


"""
    sumTorqueOverSurface_shadowing(shape, F☉, r̂☉) -> τ

# Parameters
- `shape` : Shape model
- `F☉`    : Solar irradiation [W/m^2]
- `r̂☉`    : Direction of the sun in the body-fixed frame (normalized)

# Return
- `τ`
"""
function sumTorqueOverSurface_shadowing(shape, F☉, r̂☉)
    τ = MVector(0., 0., 0.)  # YORP torque

    for facet in shape.facets
        Ψ = facet.normal ⋅ r̂☉  # cosine of the Sun illumination angle
        if Ψ > 0  # daytime hemisphere of the body
            if Astroshaper.isIlluminated(facet, r̂☉, shape.facets)
                df = Ψ * facet.area * facet.normal  # force on each facet
                dτ = facet.center × df              # torque on each facet
                τ .+= dτ
            end
        end
    end
    τ *= - 2/3 * F☉ / Astroshaper.c₀
    return SVector(τ)
end


"""
    getNetTorque(shape, orbit, spin, times) -> τ̄

Average YORP torque over given time steps
"""
function getNetTorque_shadowing(shape, orbit, spin, times)
    τ̄ = MVector(0., 0., 0.)  # net YORP torque

    for time in times
        spin_phase = spin.ω * time
        F☉, r̂☉ = getSolarCondition(orbit, spin, time)
        τ = sumTorqueOverSurface_shadowing(shape, F☉, r̂☉)
        τ = Astroshaper.body_to_orbit(τ, spin.γ, spin.ε, spin_phase)

        τ̄ .+= τ
    end
    τ̄ /= length(times)
end

getNetTorque_shadowing

In [77]:
orbit = Astroshaper.OrbitalElements(Astroshaper.RYUGU)

--------------------
  Orbital elements  
--------------------
    Semi-mojor axis         : a  = 1.18956373 [AU]
    Eccentricity            : e  = 0.19027921 [-]
    Lon. of ascending node  : Ω  = 251.589203 [deg]
    Argument of periapsis   : ω  = 211.435963 [deg]
    Inclination             : I  = 5.8840222 [deg]
    Periapsis passage time  : tₚ = -2.49480381575583e6 [sec]
    Mean anomaly            : Φ  = 21.9353799 [deg]
--------------------
  Other parameters  
--------------------
    Gravitational parameter : μ = 1.32712440018e20 [m^3/s^2]
    Mean motion             : n = 0.7596656744674016 [deg/day]
    Orbital period          : P = 473.8926768705121 [day]
------------------
  Time-variables  
------------------
    Time                    : t  = 0.0 [sec]
    Eccentric anomaly       : u  = 26.861352366162944 [deg]
    True anomaly            : ν  = 32.29438904892344 [deg]
    Position                : r  = [1.2489380764712772e11, 7.893747000217802e10, 0.0] [m]
    Velocity

In [78]:
spin = Astroshaper.SpinParams(Astroshaper.RYUGU, orbit)

-------------------
  Spin parameters  
-------------------
Right ascension (RA) : α = 96.4 [deg]
Declination (Dec)    : δ = -66.4 [deg]
Ecliptic longitude   : λ = 179.33757188938182 [deg]
Ecliptic latitude    : β = -87.44207056697601 [deg]
Obliquity            : ε = 171.65664649001607 [deg]
Spin period          : P = 7.63262 [hours]
Spin rate            : ω = 0.00022866712242903872 [rad/sec]
Vernal equinox lon.  : γ = 154.3323208882217 [deg]
                           (longitude from the periheion direction)
Time                 : t  = 0.0 [sec]
Initial spin phase   : ϕ₀ = 0.0 [deg]
Spin phase           : ϕ  = 0.0 [deg]


In [81]:
Δt = spin.P / 72
times = collect(0:Δt:orbit.P);

YORP torque averaged over an orbit cycle

In [82]:
τ̄ = getNetTorque_shadowing(shape, orbit, spin, times)

3-element MVector{3, Float64} with indices SOneTo(3):
  3.293794071192184
 -1.8096012087962008
  0.3083406885385439

In [83]:
MOI = 4.039541372643629e16  # Moment of inertia

## Orbitally averaged torque [N ⋅ m]
τ̄_ω = Astroshaper.getτω(τ̄, spin)
τ̄_ε = Astroshaper.getτε(τ̄, spin)
τ̄_ψ = Astroshaper.getτψ(τ̄, spin)

## Acceleration rate [rad/sec/sec]
ω̇  = τ̄_ω / MOI
ωε̇ = τ̄_ε / MOI
ωψ̇ = τ̄_ψ / MOI

## Acceleration rate [deg/day/day]
ω̇  = rad2deg(ω̇)  * (3600*24)^2
ωε̇ = rad2deg(ωε̇) * (3600*24)^2
ωψ̇ = rad2deg(ωψ̇) * (3600*24)^2;

In [88]:
@show ω̇
@show ωε̇
@show ωψ̇;

ω̇ = -3.544108293602548e-6
ωε̇ = 1.6667681590116214e-6
ωψ̇ = 3.973287955614437e-5


See Table 3 in Kanamaru et al. (2021)

(ω̇, ωε̇, ωψ̇) = (-3.531e-6, 1.667e-6, 3.973e-5)